In [1]:
import re
import sys
import datetime
sys.path.append(r"D:\往期\QHJ\ZERO")
sys.path.append(r"E:\dataparse\Python_DATA_PARSE\QHJ\ZERO")
from pyquery import PyQuery as pq
import requests
from urllib.parse import urlencode,parse_qs
import numpy as np
import pandas as pd
import redis

import json
from tools import *



# 测试
# 测试
# conn = redis.Redis(**REDIS_TEST_DIC)


conn = redis.Redis(**REDIS_MALL_ORDER_DIC)

In [2]:
l = []
data = pd.read_excel(r"C:\Users\zero\Desktop\TT\趣领运货单号12-13.xlsx",converters={"订单号":str})
l.append(data)
data = pd.read_excel(r"C:\Users\zero\Desktop\TT\趣领运货单号12-14.xlsx",converters={"订单号":str})
l.append(data)
data = pd.concat(l)

In [8]:
len(data)

25

In [10]:
headers = {
    "accept": "*/*",
"accept-encoding": "gzip, deflate, br",
"accept-language": "zh-CN,zh;q=0.9,en;q=0.8",
"cache-control": "no-cache",
# "content-length": "53",
"content-type": "application/x-www-form-urlencoded; charset=UTF-8",
"cookie": "PHPSESSID=m442n8rq407kg529gn37tf4vkd",
"origin": "https://app0001.yrapps.cn",
"pragma": "no-cache",
"referer": "https://app0001.yrapps.cn/admin/order/order_delivery/order_id/7801733718286446.html",
"sec-fetch-mode": "cors",
"sec-fetch-site": "same-origin",
"user-agent": "Mozilla/5.0 (Windows NT 6.1; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/78.0.3904.108 Safari/537.36",
"x-requested-with": "XMLHttpRequest",
}

url = r"https://app0001.yrapps.cn/admin/order/send_order.html"



In [12]:
data.columns

Index(['订单号', '商品ID', '商品名', '数量', '规格', '单位', '收件人', '收货地址', '联系方式', '运单号',
       '快递方式'],
      dtype='object')

In [13]:
for i,row in data.iterrows():

    data = {
        "express_info": row['快递方式'],
        "express_id": row['运单号'],
        "order_id": row['订单号']
    }
    print(data)
    requests.post(url,headers=headers,data=data)
    
#     requests.post()
    
    
    

{'express_info': '中通快递', 'express_id': 75320470374422, 'order_id': 5609222359417374}
{'express_info': '中通快递', 'express_id': 75320470374435, 'order_id': 9784424719606776}
{'express_info': '中通快递', 'express_id': 75320470374448, 'order_id': 3400138912332780}
{'express_info': '中通快递', 'express_id': 75320470374456, 'order_id': 7620266221650637}
{'express_info': '中通快递', 'express_id': 75320470374463, 'order_id': 8171817913572174}
{'express_info': '中通快递', 'express_id': 75320470374476, 'order_id': 2758468066946996}
{'express_info': '中通快递', 'express_id': 75320470374514, 'order_id': 501250902978832}
{'express_info': '中通快递', 'express_id': 75320470374489, 'order_id': 2163778663397136}
{'express_info': '中通快递', 'express_id': 75320470374491, 'order_id': 3960831208197543}
{'express_info': '中通快递', 'express_id': 75320470374491, 'order_id': 1554653366651438}
{'express_info': '中通快递', 'express_id': 75320470374501, 'order_id': 9076677957182044}
{'express_info': '中通快递', 'express_id': 75320470374514, 'order_id':

In [9]:

for i,row in data.iterrows():
    r= conn.sismember('order:new',row['订单号'])
    if not r:
        print(row['订单号'])
    else:
        print(r)

True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True


In [ ]:
stop


三金_新订单 2019-12-12 15_37_45.xlsx
三金_新订单 2019-12-13 15_38_09.xlsx
三金_新订单 2019-12-14 15_46_34.xlsx
三金_新订单 2019-12-15 15_36_53.xlsx


In [ ]:
for i,row in data.iterrows():
    spp_id = row['商品ID'][:5]
    print(row['单号'])
    r = conn.smove("order:old:%s" %(spp_id),
               "ORDER:RC:%s" %(spp_id)
               ,row['订单号'])
    print(r)

In [18]:
data = pd.read_excel(r"E:\qhj\三金外发\三金_新订单 2019-12-12 15_37_45.xlsx",index=False)
conn.sadd("order:old",*data['订单号'].tolist())
data = pd.read_excel(r"E:\qhj\三金外发\三金_新订单 2019-12-13 15_38_09.xlsx",index=False)
conn.sadd("order:old",*data['订单号'].tolist())


20

In [12]:
data['key']

0     7801733718286446_S0003
1       19121251101531_S0003
2       19121248494810_S0003
3       19121253985148_S0003
4       19121249485597_S0003
5       19121252575210_S0003
6       19121257485710_S0003
7       19121298101101_S0003
8       19121210253569_S0003
9       19121297579856_S0003
10      19121298574910_S0003
11      19121297101535_S0003
12      19121249971025_S0003
13      19121250575757_S0003
14      19121248101504_S0003
15      19121255999954_S0003
16      19121210210051_S0003
17      19121251101505_S0003
18      19121210052989_S0003
19      19121250985210_S0003
20      19121299971024_S0003
21      19121210210010_S0003
0     5609222359417374_S0003
1     9784424719606776_S0003
2     3400138912332780_S0003
3     7620266221650637_S0003
4     8171817913572174_S0003
5     2758468066946996_S0003
6      501250902978832_S0003
7     2163778663397136_S0003
               ...          
10    9076677957182044_S0003
11    1015787224552650_S0003
12     149891054111246_S0003
13    26849461

In [19]:

f_dir = r"D:\奇货居\work\外发订单\新订单\已发"
l = []
for file in os.listdir(f_dir):
    if file.startswith("三金"):
        print(file)
        file_path = os.path.join(f_dir,file)
        l.append(pd.read_excel(file_path))

data = pd.concat(l)

# data = pd.read_excel(r"D:\奇货居\work\外发订单\新订单\已发\三金_新订单 2019-12-15 15_36_53.xlsx")
# data['key'] = data['订单号'].astype(str) + "_S0003"

conn.sadd("order:old",*data['订单号'].tolist())



三金_新订单 2019-12-12 15_37_45.xlsx
三金_新订单 2019-12-13 15_38_09.xlsx
三金_新订单 2019-12-14 15_46_34.xlsx
三金_新订单 2019-12-15 15_36_53.xlsx


62